In [1]:
import json
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import numpy as np

In [ ]:
df = pd.read_csv("../processed/wikipedia_20220620_cleaned_v2.csv")
train = json.load(open("../za-data/zac2022_train_merged_final.json"))

In [5]:
for x in train['data']:
    x['dirty_text'] = None

In [6]:
all_titles = dict([(x.strip(),True) for x in open("../za-qa/wikipedia_20220620_all_titles.txt").readlines()])

In [7]:
import json 
from glob import glob 
import re 
from nltk import word_tokenize as lib_tokenizer 
 
dict_map = dict({}) 
 
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 
def strip_answer_string(text): 
    text = text.strip() 
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] != '(' and text[-1] == ')' and '(' in text: 
            break 
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1: 
            break 
        text = text[:-1].strip() 
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1: 
            break 
        text = text[1:].strip() 
    text = text.strip() 
    return text 
 
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

In [8]:
# df.question = df.question.apply(lambda x: " ".join(word_tokenize(strip_context(x))))

In [9]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [10]:
df["cleaned_text"] = df.text.parallel_apply(lambda x: " ".join(word_tokenize(strip_context(x))))

In [11]:
train_titles = set([x['title'].strip() for x in train['data'] if len(x['title']) > 0])

In [12]:
df["valid"] = df.title.parallel_apply(lambda x: str(x).strip() in train_titles)

In [13]:
df = df[df.valid]
df = df.reset_index(drop=True)

In [14]:
df.title.unique().shape

(8782,)

In [15]:
train_df = pd.DataFrame()
train_df["id"] = [x['id'] for x in train['data']]
train_df["title"] = [x['title'] for x in train['data']]
train_df["text"] = [x['text'] for x in train['data']]

In [16]:
def find_dirty_text(title, text):
    text= " ".join(word_tokenize(strip_context(text)))
    if text[-1] == ".":
        text = text[:-1].strip()
    tmp = df[df.title == title].reset_index(drop=True)
    if len(tmp) == 0:
        # print(title)
        return None
    for candidate in tmp.cleaned_text:
        if text in candidate:
            # print("Found dirty")
            return candidate
    return text

In [17]:
train_df["dirty_text"] = train_df.parallel_apply(lambda row: find_dirty_text(row.title, row.text),axis=1)

In [18]:
train_df[train_df.dirty_text.isna()]

,id,title,text,dirty_text
1,c926e7b0717202618a10dd907d4b4c39,,"có 23 quốc gia không có lực lượng quân đội, ba...",None
2,d38ef5bf1fb82b410026ed82c8a44cae,Raymondienne,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...,None
14,361a30769c1a5dca2a7b8f5c7f601982,Bánh gai Tứ Trụ,"Làng Mía thuộc hữu ngạn sông Chu, cách thị trấ...",None
18,5d7d3b0d5cd7b2917a2081c3b4d7c8a8,Nhà nước Hồi giáo ( chính thể ),Khái niệm về nhà nước Hồi giáo hiện đại đã đượ...,None
20,d7000d411d3f04dee7fa327ef11e3db0,Quan hệ Israel – Liban,Thủ tướng Liban Fouad Siniora cho biết vào thá...,None
...,...,...,...,...
20804,cb4d3b162a56b52a4927ab29b739c7a2,Thủy điện An Khê - Kanak,Nhà máy được khởi công xây dựng vào ngày 14 th...,None
20834,d701dfcba59420604c66e803c9556754,Sao Hoả,"Sao Hoả còn gọi là : Hoả Tinh , ( Tiếng Anh : ...",None
20836,5769cee0487a9674306d271854244b39,,Máy quay phim: Phát minh kỳ diệu của anh em Lu...,None
20843,5f420b6d68436b684f33dcded61d5a3c,Egil Kapstad,"Ông sinh ra tại Oslo, thủ đô của Na Uy và thàn...",None


In [19]:
dirty_text_dict = dict([x,y] for x,y in zip(train_df.id, train_df.dirty_text))

In [20]:
for x in train['data']:
    x['dirty_text'] = dirty_text_dict.get(x['id'],x['text'])

In [21]:
json.dump(train,open("../za-data/zac2022_train_merged_final.json","wt"))